In [2]:
!pip install keras-tuner

In [6]:
# Import our dependencies
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf

#read in data for tuner
finaldf = pd.read_csv("./Data/FinalData.csv")
#drop catagorical columns
finaldf1 = finaldf.copy()
finaldf1 = finaldf1.drop(columns=["Contract Number","Organization Type","Contract Name","Organization Marketing Name","Parent Organization"])
#add dummies
finaldf1 = pd.get_dummies(finaldf1)
finaldf1['Standardized Enrollment'] = finaldf1['Standardized Enrollment']-1

# Separate the y variable
y = finaldf1["Standardized Enrollment"]
# Separate the X variable, the features
X = finaldf1.drop(columns="Standardized Enrollment")

# Use sklearn to split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid','leaky_relu'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=417))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [9]:
# Import the keras-tuner library
import keras_tuner as kt
tuner = kt.Hyperband(
    create_model, 
    seed=42,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    directory="tune_dir",
    project_name="tensorworld",)

c:\Users\samsi\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 07s]
val_accuracy: 0.7129963636398315

Best val_accuracy So Far: 0.7265343070030212
Total elapsed time: 00h 04m 09s


In [11]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 1,
 'num_layers': 4,
 'units_0': 1,
 'units_1': 9,
 'units_2': 9,
 'units_3': 3,
 'units_4': 7,
 'units_5': 7,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [12]:
# Evaluate best model against full test data
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

35/35 - 0s - 8ms/step - accuracy: 0.7265 - loss: 0.2175
Loss: 0.21748434007167816, Accuracy: 0.7265343070030212
